# Plotting data on a Smith chart

This notebook focuses on **common plotting workflows**:

- Plotting points and sweeps (`ax.plot`, `ax.scatter`)
- Styling traces (markers, lines)
- Labels and legends
- Practical patterns for frequency sweeps
- Using `ax.annotate` and `ax.text` effectively

For domain selection and normalization details, see **smith_chart_domains.ipynb**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import pysmithchart
from pysmithchart.constants import IMPEDANCE_DOMAIN, ABSOLUTE_DOMAIN, REFLECTANCE_DOMAIN
from pysmithchart import utils

## 1. Create the axes (recommended pattern)

Throughout the docs we use:

```python
plt.figure(figsize=(6, 6))
ax = plt.subplot(111, projection="smith", ...)
```

In [ ]:
plt.figure(figsize=(6, 6))
sc = {
    "Z0": 50 Ω,
}
ax = plt.subplot(111, projection="smith", projection="smith", **sc)
ax.set_title("Blank Smith chart")
plt.show()

## 2. Points: `plot()` vs `scatter()`

- `ax.plot(...)` is convenient for lines and markers, and integrates well with legends.
- `ax.scatter(...)` is convenient when you want point-specific sizes or colors.

In both cases, you can pass complex data directly.

In [ ]:
Z0 = 50
Z_points = np.array([Z0 + 0j, 2 * Z0 + 0j, Z0 + 1j * Z0, Z0 - 1j * Z0])

plt.figure(figsize=(6, 6))
sc = {
    "Z0": Z0,
}
ax = plt.subplot(111, projection="smith", Z0=Z0, projection="smith", **sc)

ax.plot(Z_points, marker="o", linestyle="", label="plot markers")
ax.scatter(Z_points * 0.7, s=80, label="scatter markers")

ax.legend(loc="upper right")
ax.set_title("Points with plot() and scatter()")
plt.show()

## 2a. What inputs does `plot()` accept?

Matplotlib (and `pysmithchart`) accept several common call patterns. All of the following are supported and widely used:

### Complex array (recommended for Smith charts)

```python
ax.plot(z_array)
ax.scatter(z_array)
```

Here `z_array` is a NumPy array of complex values (e.g., `Z` in ohms for the default impedance domain).

### Separate real and imaginary arrays

```python
ax.plot(x_array, y_array)
ax.scatter(x_array, y_array)
```

Here `x_array` and `y_array` are real-valued arrays (e.g., `x = np.real(Z)`, `y = np.imag(Z)`), of the same length.

### Single complex point vs single (x, y) point

```python
ax.plot(z)      # z is complex scalar
ax.plot(x, y)   # x and y are real scalars
```

For single points, remember that `plot()` only draws markers if a marker is specified (see next section).

In [ ]:
Z0 = 50
Z = np.array([Z0 + 0j, Z0 + 1j * Z0, 2 * Z0 + 1j * Z0])

plt.figure(figsize=(6, 6))
sc = {
    "Z0": Z0,
}
ax = plt.subplot(111, projection="smith", Z0=Z0, projection="smith", **sc)

# 1) complex array
ax.plot(Z, label="ax.plot(Z)")

# 2) separate arrays
ax.plot(Z.real, Z.imag, linestyle="--", label="ax.plot(Z.real, Z.imag)")

ax.legend(loc="upper right")
ax.set_title("Equivalent plot() call patterns")
plt.show()

## 2b. Points vs lines in `plot()` (format strings, markers, and line styles)

`ax.plot()` can draw **lines**, **markers**, or both. The behavior depends on the format string and/or keyword arguments:

- `'b'` draws a **blue line**. If you pass only a single point, there is no segment to draw, so you may see nothing.
- `'bo'` draws **blue circle markers** (and no line if you also use `linestyle=""`).
- `'b-'` draws a blue **solid line**.
- `'b--'` draws a blue **dashed line**.
- `'o'` draws circle markers (default color cycle).

Recommended explicit style controls:

- marker size: `ms=10` is the same as `markersize=10`
- line style: `ls="--"` is the same as `linestyle="--"`
- to draw markers only: `linestyle=""` (or `ls=""`)
- to draw line only: omit `marker` (or set `marker=""`) and keep a line style

The examples below illustrate a few common patterns.

In [ ]:
plt.figure(figsize=(6, 6))
sc = {
    "Z0": 50,
}
ax = plt.subplot(111, projection="smith", Z0=50, projection="smith", **sc)

# A short trace
Z_trace = np.array([50 + 0j, 50 + 1j * 50, 100 + 1j * 50])

# line + markers (explicit)
ax.plot(Z_trace, marker="o", markersize=8, linestyle="-", label="line + markers")

# markers only (format string + ls="")
ax.plot(Z_trace, "bo", ls="", ms=10, label="markers only: 'bo' + ls=''")

# line only (no marker)
ax.plot(Z_trace, "b--", label="line only: 'b--'")

# single point: needs a marker to be visible
ax.plot([75 + 1j * 25], "ro", ms=10, label="single point: 'ro'")

ax.legend(loc="upper right")
ax.set_title("plot(): controlling markers vs lines")
plt.show()

## 2c. Marker control with `scatter()`

`ax.scatter()` is always **marker-based** (it does not draw connected line segments).
Compared with `plot()`, the common controls are different:

- size is `s=...` (marker area in points²), not `markersize=...`
- marker shape is `marker="o"` (or `"s"`, `"^"`, etc.)
- you can set transparency with `alpha=...`

Use `scatter()` when you want:
- purely point-based plots
- point-by-point size differences
- very large datasets where you do not want connecting lines

In [ ]:
plt.figure(figsize=(6, 6))
sc = {
    "Z0": 50,
}
ax = plt.subplot(111, projection="smith", Z0=50, projection="smith", **sc)

Z_pts = np.array([50 + 0j, 100 + 0j, 50 + 1j * 50, 50 - 1j * 50])
sizes = [40, 120, 80, 80]

ax.scatter(Z_pts, s=sizes, marker="o", alpha=0.9, label="scatter points")

# If you want a line through the same points, add plot() separately
ax.plot(Z_pts, linestyle="--", label="connecting line (plot)")

ax.legend(loc="upper right")
ax.set_title("scatter(): point control differs from plot()")
plt.show()

## 3. Traces: plotting a sweep

A very common workflow is a **frequency sweep**, where the impedance changes with frequency.

Below is a simple example using a **series RLC** model to generate a complex impedance sweep:

\[ Z(\omega) = R + j\left(\omega L - \frac{1}{\omega C}\right) \]

This example is not meant to be a circuit tutorial; it just produces a realistic sweep shape.

In [ ]:
# Series RLC example impedance sweep
R = 25.0  # ohms
L = 150e-9  # H
C = 2.0e-12  # F

f = np.logspace(7, 10, 401)  # 10 MHz to 10 GHz
w = 2 * np.pi * f
Z = R + 1j * (w * L - 1 / (w * C))

plt.figure(figsize=(6, 6))
sc = {
    "Z0": Z0,
}
ax = plt.subplot(111, projection="smith", Z0=Z0, projection="smith", **sc)

ax.plot(Z, label="Series RLC sweep")
ax.legend(loc="upper right")
ax.set_title("Impedance sweep (ohms, IMPEDANCE_DOMAIN)")
plt.show()

### `text()` vs `annotate()`

- `ax.text(x, y, "label")` places text at a location (no arrow).
- `ax.annotate("label", xy=(...), xytext=(...), arrowprops=...)` is best when you want to clearly point to a feature.

Both methods use the **same domain rules** as plotting. If you ever need to be explicit, pass `domain=...` to the call.

#### Placing text independent of the Smith-chart domain

Sometimes you want text or annotations that are **anchored to the chart frame**, not to impedance, Γ, or any other domain quantity.
In that case, use Matplotlib's axes coordinate system:

```python
ax.text(0.05, 0.95, "Annotation text",
        transform=ax.transAxes,
        ha="left", va="top")
```

Key points:

- `transform=ax.transAxes` uses **axes coordinates**, not Smith-chart data coordinates.
- `(0, 0)` is the lower-left of the chart area; `(1, 1)` is the upper-right.
- This works the same way regardless of the active Smith-chart domain.

This is ideal for titles, notes, legends, or explanatory text that should not move when the plotted data changes.

### Labeling key frequencies

A readable chart usually labels just a few key points (start/middle/end or resonance).
Here we annotate the first, middle, and last frequency points.

In [ ]:
plt.figure(figsize=(6, 6))
sc = {
    "Z0": Z0,
}
ax = plt.subplot(111, projection="smith", Z0=Z0, projection="smith", **sc)

ax.plot(Z, label="Series RLC sweep")

idxs = [0, len(f) // 2, -1]
for i in idxs:
    ax.plot([Z[i]], marker="o", linestyle="")
    ax.annotate(
        f"{f[i]/1e9:.2f} GHz",
        xy=(Z[i].real, Z[i].imag),
        xytext=(Z[i].real + 8, Z[i].imag + 8),
        arrowprops=dict(arrowstyle="->"),
    )

ax.legend(loc="upper right")
ax.set_title("Annotating points on a sweep")
plt.show()

## 4. Plotting in Γ (reflection coefficient)

If your data source provides **S11 / Γ**, you can plot it directly using `REFLECTANCE_DOMAIN`.

Reminder: in this domain, values must satisfy \(|\Gamma| \le 1\) to appear on the chart.

In [ ]:
# Convert the RLC sweep to Γ using Z0, then plot in REFLECTANCE_DOMAIN
Gamma = utils.calc_gamma(Z0, Z)

plt.figure(figsize=(6, 6))
sc = {
    "domain": REFLECTANCE_DOMAIN,
}
ax = plt.subplot(111, projection="smith", domain=REFLECTANCE_DOMAIN, projection="smith", **sc)

ax.plot(Gamma, label="Γ sweep")
ax.legend(loc="upper right")
ax.set_title("Same sweep plotted as Γ (REFLECTANCE_DOMAIN)")
plt.show()

## 5. Plotting with normalized impedance (ABSOLUTE_DOMAIN)

If you already normalized your data (e.g., in a homework problem), use `ABSOLUTE_DOMAIN` and pass \(z = Z/Z_0\).

In [ ]:
z = Z / Z0

plt.figure(figsize=(6, 6))
sc = {
    "domain": ABSOLUTE_DOMAIN,
}
ax = plt.subplot(111, projection="smith", domain=ABSOLUTE_DOMAIN, projection="smith", **sc)

ax.plot(z, label="normalized z sweep")
ax.legend(loc="upper right")
ax.set_title("Sweep plotted as normalized z (ABSOLUTE_DOMAIN)")
plt.show()

## 6. Legends and readability

A few practical tips for readable Smith charts:

- Use **one main trace** and only a few highlighted markers.
- Prefer short labels (e.g., “10 MHz”, “1 GHz”, “10 GHz”).
- Avoid placing text directly on top of the trace; offset text slightly.
- When you need directionality, use arrows (covered in `arrows_and_paths.ipynb`).

## Summary

- Use the explicit axes creation pattern:

  ```python
  plt.figure(figsize=(6, 6))
  ax = plt.subplot(111, projection="smith", Z0=50)
  ```

- Pass complex values directly to `plot()` / `scatter()`.
- Use `annotate()` with arrows to label a small number of points.
- Choose `domain=` based on your data source (Z, z, or Γ).

Next recommended notebook: **vswr_and_circles**.